# Lab 1:  Data Loading, GPU Dataframe Creation, and Data Manipulation
Thanks to Anaconda for some material

In this lab you will learn how to 
- Load data into a GPU Data Frame (GDF)
- Learn about Dataframe level data functions
- Learn about Seriers level data function 
- Manitulate data in a GDF to performance some basic ETL and statistical functions


For this lab we will be looking at Netflow data.  The data consist of seven column which make it easy to visually inspect.  We will be manipulating the data in various way, many of which make not sense for doing cyber analysis.  


This lab should take 30 to 50 minutes


<br>
As you progress in this lab look for instances of ***TASK:***, this will be where you will be asked to take an action to complete this lab.

***TASK:*** Execute the cell below to load the auto-time modules so that runtime execution of every cell is reported

In [ ]:
# Add autotime of each block
#!pip install ipython-autotime
%load_ext autotime

# Loading Data
The first file we are going to load is netflow1.csv
  
Size = 965 MB<br>
Records = 17,296,829

### Load using the traditional Pandas interface

In [ ]:
import pandas as pd

In [ ]:
!ls /dli/data/kdd-data

In [ ]:
!ls /dli/data/kdd-data/netflow

In [ ]:
# specify the file name
file_1 = '/dli/data/kdd-data/netflow/netflow1.csv'

In [ ]:
# let's define the data - column names and data types 
cols = [
    "strdate",
    "srcip",
    "dstip",
    "srcport",
    "dstport",
    "srcbytes",
    "dstbytes"   
]


dtypes = {    
    "strdate"  : str,
    "srcip"    : "category",
    "dstip"    : "category",
    "srcport"  : int,
    "dstport"  : int,
    "srcbytes" : int,
    "dstbytes" : int

}

In [ ]:
# the data file contains a header line that needs to be skipped
df = pd.read_csv(file_1,  names=cols, dtype=dtypes, skiprows=1)

In [ ]:
# look at the loaded data types
df.dtypes

In [ ]:
# look at the first few rows of data
df.head()

<br>
# Creating a GPU Dataframe


In [ ]:
import pygdf

In [ ]:
# Strings aren't supported yet
gdf = pygdf.DataFrame.from_pandas(df.drop(["strdate"], axis=1))

In [ ]:
gdf.dtypes

In [ ]:
# let's look at the data
print(gdf)

## ***Congrats you have created a GPU Data Frame!!!***

### Transitioning GDF back to Pandas

In [ ]:
# just as easy
df2 = gdf.to_pandas()

In [ ]:
type(df2)

### Clean-up: deleting a dataframe

In [ ]:
# clean up since we don't need df2
del(df2)

In [ ]:
# Also drop the GPU dataframe since we will recreate it with some options
del(gdf)

In [ ]:
del(df)

# Column-based funtions and Data Manipulation

In [ ]:
# reload the data
df = pd.read_csv(file_1,  names=cols, dtype=dtypes, skiprows=1)

### Limitations on the current GDF version
- Date are not fully supported   => scheduled for next version
- String are not supported => also scheduled for next version
- Categoricals are in early alpha where issues are expected

In [ ]:
# could do df['strdate'] or ..
df.strdate.head(2)

In [ ]:
# Within Pandas - Convert the data from a string into parts
df['Date']  = pd.to_datetime(df['strdate'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
df['Year']  = df.Date.dt.year
df['Month'] = df.Date.dt.month

***Task:***  Convert day, hour, min, and second
<details><summary>Click for Answer</summary>
<code>
df['Day']  = df.Date.dt.day
df['Hour'] = df.Date.dt.hour
df['Minute']  = df.Date.dt.minute
df['Second'] = df.Date.dt.second
</code>
</details>

In [ ]:
# Your code here


In [ ]:
df.dtypes

In [ ]:
# drop the columns we no longer care about
df = df.drop(['strdate','Date'], axis=1)

In [ ]:
# now create the GDF one more time
gdf = pygdf.DataFrame.from_pandas(df)

In [ ]:
gdf.dtypes

That's it - the data is now loaded and available for accelerated Data Analysis on the GPU
<br>
<br>
*Remember that **df = CPU** and **gdf = GPU** since we will switch back and forth for performance comparisons*
<br>
<br>
***Data Loading Runtime*** = pd.read_csv + DataFrame.from_pandas

# Column Functionals and Transformations
One of the basic GDF operations is column transform. To do that we use built-in arithmetic operations on each column

***Note:*** The followiong function operate against a GDF Column - not against the full dataset.


### Simple Counting

In [ ]:
# First some simple counting of the number of rows
# NOTE: this counts the number of ***Non-NULL*** records are in the dataset?  
gdf['srcport'].count()

***Task:***  How long does it take to do that simple counting on the CPU?
<details><summary>Click for Answer</summary>
<code>
df['dstbytes'].count()
</code>
</details>

In [ ]:
# Your code here


In [ ]:
### That is not enough data to truely push a GPU, let's load more
file_2 = '/dli/data/kdd-data/netflow/netflow2.csv'
#file_3 = '/dli/data/kdd-data/netflow/netflow3.csv'
#file_4 = '/dli/data/kdd-data/netflow/netflow4.csv'

df2 = pd.read_csv(file_2,  names=cols, dtype=dtypes, skiprows=1)
#df3 = pd.read_csv(file_3,  names=cols, dtype=dtypes, skiprows=1)
#df4 = pd.read_csv(file_4,  names=cols, dtype=dtypes, skiprows=1)

In [ ]:
def ExtractDateTime (d, column) :
    d['Date']   = pd.to_datetime(d[column], format='%Y-%m-%d %H:%M:%S')
    d['Year']   = d.Date.dt.year
    d['Month']  = d.Date.dt.month    
    d['Day']    = d.Date.dt.day
    d['Hour']   = d.Date.dt.hour
    d['Minute'] = d.Date.dt.minute
    d['Second'] = d.Date.dt.second   
    
    d = d.drop([column,'Date'], axis=1)
    
    return d

In [ ]:
df2 = ExtractDateTime(df2, 'strdate')
#df3 = ExtractDateTime(df3, 'strdate')
#df4 = ExtractDateTime(df4, 'strdate')

In [ ]:
gdf2 = pygdf.DataFrame.from_pandas(df2)
#gdf3 = pygdf.DataFrame.from_pandas(df3)
#gdf4 = pygdf.DataFrame.from_pandas(df4)

In [ ]:
#There's currently a bug in PyGDF in concatenating dataframes or Series containing "category" dtypes,
#so lets drop those columns

#Remove GDF columns is different than Pandas - can only do one column at a time for now
gdf.drop_column('srcip')
gdf.drop_column('dstip')

gdf2.drop_column('srcip')
gdf2.drop_column('dstip')

#gdf3.drop_column('srcip')
#gdf3.drop_column('dstip')

#gdf4.drop_column('srcip')
#gdf4.drop_column('dstip')

In [ ]:
# Concatenate the GDFs together
#gdf = pygdf.concat([gdf, gdf2, gdf3, gdf4], ignore_index=True)
gdf = pygdf.concat([gdf, gdf2], ignore_index=True)
del(gdf2)
#del(gdf3)
#del(gdf4)

In [ ]:
gdf['srcport'].count()

was 17,296,828 now is 35,319,196

In [ ]:
# do the same on the CPU so that the two datasets are in sync
#df = pd.concat([df, df2, df3, df4])
df = pd.concat([df, df2])

In [ ]:
# and do a count
df['srcport'].count()

## Data Type Conversion
Let's continue cleaning up the data

### What is the max value for the source port?

In [ ]:
gdf['srcport'].max()

In [ ]:
# Since the largest dstbyte size is less than an int32, let's convert the data type
# This matches the Pandas syntax
import numpy as np
gdf['srcport'] = gdf['srcport'].astype(np.int32)

In [ ]:
gdf.dtypes

***Tasks:*** Find the max values for dstport, srcbytes, and dstbytes

<details><summary>Click for Answer</summary>
<code>
gdf['dstport'].max()
gdf['srcbytes'].max()
gdf['dstbytes'].max()
</code>
</details>

In [ ]:
# Your code here


***Tasks:*** Convert dstport to int32

<details><summary>Click for Answer</summary>
<code>
gdf['dstport'] = gdf['dstport'].astype(np.int32)
</code>
</details>

In [ ]:
# Your code here


***Question:*** Why do we care about using smaller data types?

***Tasks:*** Looking at dstbytes:  what is min, max, mean, and standard deviation?

<details><summary>Click for Answer</summary>
<code>
gdf['dstbytes'].min()
gdf['dstbytes'].max()
gdf['dstbytes'].mean()
gdf['dstbytes'].std()
</code>
</details>

In [ ]:
# Your code here


## Now Let's create a new column

### Transformations
***Tasks:*** Create a new GDF column called **`totalbytes`** that is the sum of src and dst bytes

<details><summary>Click for Answer</summary>
<code>
gdf['totalbytes'] = gdf['srcbytes'] + gdf['dstbytes']
</code>
</details>

In [ ]:
# Your code here


In [ ]:
# Verify that a new column was created
gdf.dtypes

In [ ]:
# What is the max byte size
gdf['totalbytes'].max()

#### Let's try that same function on the CPU

In [ ]:
# See the same time on the CPU
df['totalbytes'] = df['srcbytes'] + df['dstbytes']

In [ ]:
df['totalbytes'].max()

The performance gain is on a small dataset and a simple transformation.  As data size and analytic complexity increas so does the delta in performance.

# DataFrame Operations
The following function operate on a dataframe

### Filtering
Selecting a subregion of data that matches an expresion

In [ ]:
# Let's get a count for reference of the total number of records
gdf['srcport'].count()

### Extract a new Dataframe where the DST Port is not port 80

In [ ]:
port_not_80 = gdf.query('dstport != 80')

***Task:***  How many records are in the new set?
<details><summary>Click for Answer</summary>
<code>
port_not_80['srcport'].count()
</code>
</details>

In [ ]:
# Your code here


In [ ]:
# Just to verify that this is a complete dataframe
print(port_not_80)

In [ ]:
# and let's do the same on the CPU
cpu_port_not_80 = df.query('dstport != 80')

***Task:***  delete those two dataset
<details><summary>Click for Answer</summary>
<code>
del(port_not_80)
del(cpu_port_not_80)
</code>
</details>


In [ ]:
# Your code here


### Sorting

In [ ]:
# Sort the data by the Source Port  
# ***Note*** current version only support sorting on a single column
by_srcport = gdf.sort_values(by='srcport')

In [ ]:
print(by_srcport)

### Grouping and Aggregations

In [ ]:
from collections import OrderedDict

In [ ]:
gdf.dtypes

In [ ]:
# add a column for count
gdf['dummy'] = gdf['dstport']
gdf.dtypes

In [ ]:
aggs = OrderedDict()
aggs['dummy'] = 'count'

stats = gdf.groupby(['dstport']).agg(aggs)

In [ ]:
stats.dtypes

In [ ]:
print(stats)

In [ ]:
del(stats)

***Tasks:***  What is the count of SRC - DST Port groupings

<details><summary>Click for answer</summary>
<code>
aggs = OrderedDict()
aggs['dummy'] = 'count'

stats = gdf.groupby(['srcport','dstport']).agg(aggs)
</code>
</details> 

In [ ]:
# Your code here
aggs = OrderedDict()
aggs['dummy'] = 'count'

stats = gdf.groupby(['srcport','dstport']).agg(aggs)

In [ ]:
print(stats)

In [ ]:
del(stats)

## Independent Task

***TASK***
- Normalize the total byte count to KB
- Compute the Mean and Standard Deviations
- Compute the Z-Score

<details><summary>Click for answer - just one possibility</summary>
<code>
def Data_to_KB( raw_bytes ) :
    return raw_bytes / 1024
</code>
<code>    
gdf['KB'] = gdf['totalbytes'].applymap(Data_to_KB)
</code>
<code>
mean = gdf['KB'].mean()
</code>
<code>
std = gdf['KB'].std()
</code>
<code>
print("Std == %f \t Mean == %f" % (std, mean))
</code>
<code>
def z_score(kb) :
    return ( (kb - mean) / std)
</code>
<code>
gdf['Z'] = gdf['totalbytes'].applymap(Data_to_KB)
</code>
<code>
gdf.dtypes
</code>
<code>
print(gdf.head())
</code>
</details> 

In [ ]:
# Your code here
